In [5]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import time

URL = "https://prod.aufgabenpool.at/srdp/phpcode/search.php"


all=[]
#loop through a to z
for i in range(97, 123):
#post request with query=buchstabe as payload
    r = requests.post(url = URL, data = {'query': chr(i)})
    #extracting response text
    soup = BeautifulSoup(r.content, "html.parser")
    #get all <li> tags
    li = soup.find_all('li')

    
    #get "id" attribute of the <div> in <li> tag
    li_id = [i.find('div')['id'] for i in li]

    #get inner text of <li> tags
    li_text = [i.text for i in li]

    #remove text between and "<" until ">" 
    li_text = [i.split("<")[0] for i in li_text]

    #remove "(number)" at the end
    li_text = [i.split("(")[0] for i in li_text]

    #trim whitespaces
    li_text = [i.strip() for i in li_text]

    #combine id and text to json
    li_text = [{"id": li_id[i], "thema": li_text[i]} for i in range(len(li_id))]

    all += li_text
    print(len(all))
    time.sleep(0.2)

#remove duplicates
all = list({v['id']:v for v in all}.values())

#remove {"id": "Ooops ...", "thema": "Ihre Suche hat leider keine Treffer erzielt."}
all = [i for i in all if i['id'] != "Ooops ..."]

#save as json
with open('/content/drive/MyDrive/themen.json', 'w') as outfile:
    json.dump(all, outfile)


37
44
68
86
133
151
183
217
261
262
284
321
348
397
432
442
445
483
509
551
587
595
602
605
608
622


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import time

URL = "https://aufgabenpool.at/srdp/phpcode/lp_results.php"

#read themen.json
with open('/content/drive/MyDrive/themen.json') as json_file:
    data = json.load(json_file)

res=[]
for el in data:
    payload = {'id': el['id'], 'coll': ''}

    #post request
    r = requests.post(url = URL, data = payload)

    #extracting response text
    soup = BeautifulSoup(r.content, "html.parser")

    #extract all <li> tags
    li = soup.find_all('li')

    #pdf is <a> tag
    #images can be found with src attribute from <img> tag

    all_imgs = []
    for i in li:
        a_tag = i.find('a')
        if a_tag:
          temp= {'loesung': "https://aufgabenpool.at/srdp/"+a_tag['href']}
        else:
          print(i)
          continue
        #get string between last / and .pdf
        temp['name'] = i.find('a')['href'].split("/")[-1].split(".pdf")[0]
        #find all <img> tags
        imgs = i.find_all('img')
        for i in range(len(imgs)):
            if i == 0:
                temp['aufgabe_head'] = "https://aufgabenpool.at/srdp/"+imgs[i]['src']
                temp['teil'] = imgs[i]['src'].split("teil")[1].split("/")[0].upper()
                temp['id'] = imgs[i]['src'].split("/")[1].split("/")[0].upper()
                temp["thema"]=el['thema']
            elif i == 1:
                temp['aufgabe_body'] = "https://aufgabenpool.at/srdp/"+imgs[i]['src']
                break
        res.append(temp)
    print(len(res))
    time.sleep(10)

#save as json
with open('/content/drive/MyDrive/beispiele.json', 'w') as outfile:
    json.dump(res, outfile)



In [2]:
!pip install PyPDF2 pdf2image pdfminer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 77.9 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140118 sha256=6e7e69cb4979757d87e188ac895364346dd00e245ea40d0d90e3a9495fae1f4d
  Stored in directory: /root/.cache/pip/wheels/1c/28/7d/f390b82bb0307deb63ff27a1474fd308ec68ee028cb9ab6283
Successfully built pdfminer


In [3]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.2/619.2 KB 41.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.68
    Uninstalling opencv-python-headless-4.7.0.68:
      Successfully uninstalled opencv-python-headless-4.7.0.68


In [4]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (123 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 129504 fi

restart runtime

In [ ]:
import json
import sys
sys.path.append('/usr/local/lib/python3.8/dist-packages')
import requests
from PIL import Image
from io import BytesIO
import PyPDF2
import os
from pdf2image import convert_from_path
import easyocr
import re
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdf2image import convert_from_path
from PIL import Image

# Loop through the pages of the PDF
def find_right_page(pdf_reader, such_string, first_match=True):
     # Initialize a variable to keep track of the number of times the string is found
    count = 0
    such_page=len(pdf_reader.pages)-1
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
    # Search for the string in the text
        match = re.search(such_string, text)
        if match:
            if first_match:
                return page_num+1
            else:
                count += 1
                if count == 2:
                    return page_num+1
    if first_match:
      return -1
    else:
      return pdf_reader.getNumPages()-1
    
def get_string_position_and_crop(pdf_file, string, page_number):
    with open(pdf_file, 'rb') as f:
        # Create a PDF resource manager object that stores shared resources
        resource_manager = PDFResourceManager()
        # Create a PDF page aggregator object
        device = PDFPageAggregator(resource_manager, laparams=LAParams())
        # Create a PDF interpreter object
        interpreter = PDFPageInterpreter(resource_manager, device)
        # Process the specific page
        for i, page in enumerate(PDFPage.get_pages(f)):
            if i == page_number-1:
                interpreter.process_page(page)
                # Receive the LTPage object for the page
                layout = device.get_result()
                # Iterate through the layout elements
                firs=True
                pz=1
                images = convert_from_path(pdf_file, first_page=page_number, last_page=page_number)
                image = images[0]
                p2=image.height
                for element in layout:
                    if hasattr(element,'get_text'):
                        if string in element.get_text().strip():
                            print(element.get_text().strip())
                            if firs:
                                firs=False
                                pz = element.bbox[1]/layout.height
                                # Convert the page to an image
                                
                                
                            else:
                                p2 = element.bbox[1]/layout.height
                                p2=image.height-p2*image.height-50


                            # Crop the image based on the y value
                image = image.crop((0, image.height-pz*image.height-90, image.width, p2))
                return image
    return None

#open themen.json
with open("/content/drive/MyDrive/"+'themen.json') as json_file:
    themen = json.load(json_file)

if not os.path.exists("/content/drive/MyDrive/Themen/"):
  os.makedirs("/content/drive/MyDrive/Themen/")

reader = easyocr.Reader(['de'], gpu=False)
for the in themen:
  #open beispiele.json
  with open("/content/drive/MyDrive/"+'beispiele.json') as json_file:
      data = json.load(json_file)
  thema=the["thema"]
  print(thema)


  #filter by thema
  data = [i for i in data if i['thema'] == thema]


      #if no thema folder make one
  if not os.path.exists("/content/drive/MyDrive/pdfs"):
      os.makedirs("/content/drive/MyDrive/pdfs")


  for bsp in data:
      aufgabe_name= bsp["name"]+"_"+bsp["id"]

      auf_dir="/content/drive/MyDrive/Themen/"+thema+"/"+aufgabe_name

      if os.path.exists(auf_dir+"/"+"loesung.jpeg"):
          continue
      
      #get "aufgabe_body"
      ab=bsp['aufgabe_body']

      #get ab as image and ocr image
      response = requests.get(ab)
      img = Image.open(BytesIO(response.content))
      #ocr image with easyocr
      
      result = reader.readtext(img)
      print(result)
      text = result[0][1]
      
      #download "loesung" as pdf
      loe=bsp['loesung']
      pdf_name= aufgabe_name+".pdf"
      #check if already downloaded
      if not os.path.exists("/content/drive/MyDrive/"+"pdfs/"+pdf_name):
          r = requests.get(loe, allow_redirects=True)
          open("/content/drive/MyDrive/"+"pdfs/"+pdf_name, 'wb').write(r.content)

      #extract bsp letter + ")"
      aufgabe = text.split(")")[0].strip()


      pdf_file = open("/content/drive/MyDrive/"+"pdfs/"+pdf_name, "rb")
      pdf_reader = PyPDF2.PdfReader(pdf_file)

      #manually type in the aufgabe letter if aufgabe is longer than 5 char
      if len(aufgabe) > 5:

          #find text before aufgabe   
          for i in range(len(pdf_reader.pages)):
              page = pdf_reader.pages[i]
              page_text = page.extract_text()
              if aufgabe[:12] in page_text:
                  #get index of aufgabe
                  index = page_text.index(aufgabe[:12])
                  au=page_text[index-10:index]
                  aufgabe=au.strip().split(" ")[-1][:-1]

      #make aufgabe_name directory in thema dir
      if not os.path.exists(auf_dir):
          os.makedirs(auf_dir)
      
      #download "aufgabe_head" as image
      ah=bsp['aufgabe_head']
      ah_name= bsp["id"]+"ah"
      ah_path= auf_dir+"/"+ah_name+".png"
      #check if already downloaded
      if not os.path.exists(ah_path):
          response = requests.get(ah)
          img = Image.open(BytesIO(response.content))
          img.save(ah_path)

      #download "aufgabe_body" as image
      ab=bsp['aufgabe_body']
      ab_name= bsp["id"]+"ab"
      ab_path= auf_dir+"/"+ab_name+".png"
      #check if already downloaded
      if not os.path.exists(ab_path):
          response = requests.get(ab)
          img = Image.open(BytesIO(response.content))
          img.save(ab_path)
      
      #extract solution

      such_string = aufgabe+"1)"
      reg=aufgabe+"1\)"
      print(such_string)
      such_page = None



      print(such_string)
      print(pdf_file)
      such_page = find_right_page(pdf_reader, reg, True)


      if such_page == -1:
          reg=aufgabe+"\)"
          such_page = find_right_page(pdf_reader, reg, False)

      print(such_page)

          

      image = get_string_position_and_crop("/content/drive/MyDrive/"+"pdfs/"+pdf_name, such_string, such_page)
      if image:
          # Save the cropped image
          image.save(auf_dir+"/"+bsp["id"]+"loesung.jpeg")
          
      pdf_file.close()





absoluter und relativer Fehler
[([[10, 24], [36, 24], [36, 52], [10, 52]], 'c)', 0.9968967910015005), ([[47, 21], [625, 21], [625, 58], [47, 58]], 'Um die Länge L des Graphen der Funktion f im Intervall', 0.8503273342019958), ([[646, 10], [976, 10], [976, 64], [646, 64]], '4; 4] abzuschätzen; berechnet', 0.32017923843571494), ([[49, 55], [963, 55], [963, 95], [49, 95]], 'man die Gesamtlänge L, der zwei strichlierten Strecken (siehe nachstehende Abbildung) .', 0.7811202396331212), ([[488, 118], [576, 118], [576, 144], [488, 144]], 'f(x) in dm', 0.9883145663603706), ([[777, 287], [845, 287], [845, 307], [777, 307]], 'X in dm', 0.49001750726048265), ([[82, 557], [742, 557], [742, 590], [82, 590]], 'Stellen Sie eine Formel zur Berechnung der Gesamtlänge L1 auf', 0.8283228524997324), ([[82, 592], [416, 592], [416, 618], [82, 618]], 'Verwenden Sie dabei r, R und h.', 0.7397276996309996), ([[47, 744], [633, 744], [633, 780], [47, 780]], 'Folgende Berechnung wird für das Fass C durchgeführt:',

In [ ]:
!pip install genanki

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 3.9 MB/s eta 0:00:00


In [ ]:
import json
import genanki


#open beispiele.json
with open('/content/drive/MyDrive/beispiele.json') as json_file:
    data = json.load(json_file)

thema="Binomialverteilung"

my_deck = genanki.Deck(
    deck_id=12345, # a unique id for the deck
    name=thema+"_RDP_scraped") # the name of the deck

img=[]
#filter by thema
data = [i for i in data if i['thema'] == thema]
c=0
for bsp in data:
    c+=1
    aufgabe_name= bsp["name"]+"_"+bsp["id"]
    auf_dir="/content/drive/MyDrive/"+thema+"/"+aufgabe_name
    ah=bsp['aufgabe_head']
    ah_name= bsp["id"]+"ah"
    ah_path= auf_dir+"/"+ah_name+".png"
    ab=bsp['aufgabe_body']
    ab_name= bsp["id"]+"ab"
    ab_path= auf_dir+"/"+ab_name+".png"
    my_card = genanki.Note(
    model=genanki.Model(
        1380120064,
  'Example',
        fields=[
            {'name': 'Image1'},
            {'name': 'Image2'},
            
            {'name': 'AnswerImage'},
            {'name': 'Answer'},
        ],
        templates=[
            {
                'name': 'Card 1',
                'qfmt': '{{Image1}}<br>{{Image2}}',
                'afmt': '{{AnswerImage}}<br>{{Answer}}',
            },
        ]
    ),
    fields=['<img src="{}">'.format(ah_name+".png"), '<img src="{}">'.format(ab_name+".png"), '<img src="{}">'.format(bsp["id"]+"loesung.jpeg"), '<a href="{}">Link</a>'.format(bsp["loesung"])])

    img.append(ah_path)
    img.append(ab_path)
    img.append(auf_dir+"/"+bsp["id"]+"loesung.jpeg")
    my_deck.add_note(my_card)

my_package = genanki.Package(my_deck)
my_package.media_files = img

#create directory if not exists
if not os.path.exists("/content/drive/MyDrive/AM_RDP_Decks"):
    os.makedirs("/content/drive/MyDrive/AM_RDP_Decks")

my_package.write_to_file("/content/drive/MyDrive/AM_RDP_Decks/"+thema+'.apkg')



